# AMLS final project

This notebook contains the work for the ELEC0134 Applied Machine Learning Systems class at UCL. The solution is implemented in Keras.

## Using Drive with Colab, typical imports

In [20]:
# Set up Google Drive for use with Colaboratory
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

# Magic calls TensorFlow 2.0 when importing
%tensorflow_version 2.x
import tensorflow as tf

# This tests whether a GPU is running
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Import Keras
from tensorflow import keras

# Check version of TensorFlow and Keras
print(tf.__version__)
print(keras.__version__)

Found GPU at: /device:GPU:0
2.1.0-rc1
2.2.4-tf


## Utilities

In [0]:
# Convert gender test generator into numpy arrays
# https://stackoverflow.com/questions/42284873/assign-imagedatagenerator-result-to-numpy-array
itr = gender_test_gen
gender_X_test, gender_y_test = itr.next()

In [0]:
# We check the average dimensions of the images in the dataset

# Code from
# https://towardsdatascience.com/image-classification-python-keras-tutorial-kaggle-challenge-45a6332a58b8

from PIL import Image

def get_size_statistics(DIR, number_of_files):
  heights = []
  widths = []
  counter = 1
  for img in os.listdir(DIR):
    path = os.path.join(DIR, img)
    print("Opening " + path + ": " + str(counter) + "/" + str(number_of_files))
    data = np.array(Image.open(path)) # PIL Image library
    heights.append(data.shape[0])
    widths.append(data.shape[1])
    counter += 1
  avg_height = sum(heights) / len(heights)
  avg_width = sum(widths) / len(widths)
  print('\n')
  print("Average Height: " + str(avg_height))
  print("Max Height: " + str(max(heights)))
  print("Min Height: " + str(min(heights)))
  print('\n')
  print("Average Width: " + str(avg_width))
  print("Max Width: " + str(max(widths)))
  print("Min Width: " + str(min(widths)))

get_size_statistics("img/", len(df))

## Creating data input pipeline

### A1, A2

In [0]:
root_dir = "/content/drive/My Drive/"
change_dir = root_dir + "dataset_AMLS_19-20/celeba"

os.chdir(change_dir)

In [23]:
# Check current directory location and contents
!pwd
!ls

/content/drive/My Drive/dataset_AMLS_19-20/celeba
img  labels.csv


In [0]:
# Import celeba data as dataframe, drop unnecessary column
df = pd.read_csv("labels.csv", sep="\t", dtype=str)

# Create separate dataframes for gender and smiling
gender = df.copy()
smiling = df.copy()

gender.drop(gender.columns[0], axis=1, inplace=True)
gender.drop(gender.columns[2], axis=1, inplace=True)

smiling.drop(smiling.columns[0], axis=1, inplace=True)
smiling.drop(smiling.columns[1], axis=1, inplace=True)

In [25]:
df.head()

,Unnamed: 0,img_name,gender,smiling
0,0,0.jpg,-1,1
1,1,1.jpg,-1,1
2,2,2.jpg,1,-1
3,3,3.jpg,-1,-1
4,4,4.jpg,-1,-1


In [26]:
gender.head()

,img_name,gender
0,0.jpg,-1
1,1.jpg,-1
2,2.jpg,1
3,3.jpg,-1
4,4.jpg,-1


In [27]:
smiling.head()

,img_name,smiling
0,0.jpg,1
1,1.jpg,1
2,2.jpg,-1
3,3.jpg,-1
4,4.jpg,-1


In [0]:
# Now, we create training and test sets for the gender and smiling datasets
from sklearn.model_selection import train_test_split

gender_train, gender_test = train_test_split(
    gender, 
    test_size=0.2,
    random_state=42
    )

smiling_train, smiling_test = train_test_split(
    smiling, 
    test_size=0.2, 
    random_state=42
    )

In [29]:
gender_train.head()

,img_name,gender
4227,4227.jpg,-1
4676,4676.jpg,1
800,800.jpg,1
3671,3671.jpg,-1
4193,4193.jpg,-1


In [30]:
print(len(gender_train))

4000


In [31]:
gender_test.head()

,img_name,gender
1501,1501.jpg,-1
2586,2586.jpg,-1
2653,2653.jpg,-1
1055,1055.jpg,-1
705,705.jpg,1


In [32]:
print(len(gender_test))

1000


In [33]:
smiling_train.head()

,img_name,smiling
4227,4227.jpg,1
4676,4676.jpg,1
800,800.jpg,-1
3671,3671.jpg,1
4193,4193.jpg,-1


In [34]:
print(len(smiling_train))

4000


In [35]:
smiling_test.head()

,img_name,smiling
1501,1501.jpg,1
2586,2586.jpg,1
2653,2653.jpg,1
1055,1055.jpg,1
705,705.jpg,-1


In [36]:
print(len(smiling_test))

1000


In [37]:
# We now create two ImageDataGenerator objects for the gender dataset:
# one for training, the other for validation
from keras.preprocessing.image import ImageDataGenerator

# https://forums.fast.ai/t/split-data-using-fit-generator/4380/4
# for validation split

# We rescale to ensure RGB values fall between 0 and 1
# We set aside 20% of the training set for validation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# We generate an image-label pair for the training set as follows
gender_train_gen = datagen.flow_from_dataframe(
    dataframe=gender_train, 
    directory="img/",
    x_col="img_name",
    y_col="gender",
    class_mode="sparse",
    target_size=(218,178),
    batch_size=32,
    subset="training"
    )

# We generate an image-label pair for the validation set as follows
gender_val_gen = datagen.flow_from_dataframe(
    dataframe=gender_train, 
    directory="img/",
    x_col="img_name",
    y_col="gender",
    class_mode="sparse",
    target_size=(218,178),
    batch_size=32,
    subset="validation"
    )

# We generate an image-label pair for the gender test set as follows
# We set batch_size = size of test set
gender_test_gen = datagen.flow_from_dataframe(
    dataframe=gender_test, 
    directory="img/",
    x_col="img_name",
    y_col="gender",
    class_mode="sparse",
    target_size=(218,178),
    batch_size=len(gender_test)
    )

Found 3200 validated image filenames belonging to 2 classes.
Found 800 validated image filenames belonging to 2 classes.
Found 1000 validated image filenames belonging to 2 classes.


In [0]:
# Create two ImageDataGenerator objects for the smiling dataset in a similar way

# https://forums.fast.ai/t/split-data-using-fit-generator/4380/4
# for validation split

# We rescale to ensure RGB values fall between 0 and 1
# We set aside 20% of the training set for validation
datagen = ImageDataGenerator(rescale=1./255,
                             validation_split=0.2)

# We generate an image-label pair for the training set as follows
smiling_train_gen = datagen.flow_from_dataframe(
    dataframe=smiling_train,
    directory="img/",
    x_col="img_name",
    y_col="smiling",
    class_mode="sparse",
    target_size=(218,178),
    batch_size=32,
    subset="training"
    )

# We generate an image-label pair for the validation set as follows
smiling_val_gen = datagen.flow_from_dataframe(
    dataframe=smiling_train,
    directory="img/",
    x_col="img_name",
    y_col="smiling",
    class_mode="sparse",
    target_size=(218,178),
    batch_size=32,
    subset="validation"
    )

# We generate an image-label pair for the gender test set as follows
# We set batch_size = size of test set
smiling_test_gen = datagen.flow_from_dataframe(
    dataframe=smiling_test, 
    directory="img/",
    x_col="img_name",
    y_col="smiling",
    class_mode="sparse",
    target_size=(218,178),
    batch_size=len(smiling_test)
    )

Found 3200 validated image filenames belonging to 2 classes.
Found 800 validated image filenames belonging to 2 classes.
Found 1000 validated image filenames belonging to 2 classes.


### B1, B2

In [0]:
root_dir = "/content/drive/My Drive/"
change_dir = root_dir + "dataset_AMLS_19-20/cartoon_set"

os.chdir(change_dir)

In [0]:
# Check current directory location and contents
!pwd
!ls

/content/drive/My Drive/dataset_AMLS_19-20/cartoon_set
img  labels.csv


In [0]:
# Import celeba data as dataframe, drop unnecessary column
df = pd.read_csv("labels.csv", sep="\t", dtype=str)

# Create separate dataframes for face shape and eye color
face_shape = df.copy()
eye_color = df.copy()

face_shape.drop(face_shape.columns[0], axis=1, inplace=True)
face_shape.drop(face_shape.columns[0], axis=1, inplace=True)

eye_color.drop(eye_color.columns[0], axis=1, inplace=True)
eye_color.drop(eye_color.columns[1], axis=1, inplace=True)

In [0]:
df.head()

,Unnamed: 0,eye_color,face_shape,file_name
0,0,1,4,0.png
1,1,2,4,1.png
2,2,2,3,2.png
3,3,2,0,3.png
4,4,0,2,4.png


In [0]:
face_shape.head()

,face_shape,file_name
0,4,0.png
1,4,1.png
2,3,2.png
3,0,3.png
4,2,4.png


In [0]:
eye_color.head()

,eye_color,file_name
0,1,0.png
1,2,1.png
2,2,2.png
3,2,3.png
4,0,4.png


In [0]:
# Now, we create training and test sets for the face shape and eye color datasets
face_shape_train, face_shape_test = train_test_split(
    face_shape, 
    test_size=0.2,
    random_state=42
    )

eye_color_train, eye_color_test = train_test_split(
    eye_color, 
    test_size=0.2, 
    random_state=42
    )

In [0]:
face_shape_train.head()

,face_shape,file_name
9254,2,9254.png
1561,1,1561.png
1670,1,1670.png
6087,2,6087.png
6669,2,6669.png


In [0]:
print(len(face_shape_train))

8000


In [0]:
face_shape_test.head()

,face_shape,file_name
6252,3,6252.png
4684,0,4684.png
1731,0,1731.png
4742,1,4742.png
4521,1,4521.png


In [0]:
print(len(face_shape_test))

2000


In [0]:
eye_color_train.head()

,eye_color,file_name
9254,0,9254.png
1561,2,1561.png
1670,3,1670.png
6087,2,6087.png
6669,3,6669.png


In [0]:
print(len(eye_color_train))

8000


In [0]:
eye_color_test.head()

,eye_color,file_name
6252,2,6252.png
4684,2,4684.png
1731,4,1731.png
4742,2,4742.png
4521,1,4521.png


In [0]:
print(len(eye_color_test))

2000


In [0]:
# We now create two ImageDataGenerator objects for the face shape dataset:
# one for training, the other for validation
from keras.preprocessing.image import ImageDataGenerator

# https://forums.fast.ai/t/split-data-using-fit-generator/4380/4
# for validation split

# We rescale to ensure RGB values fall between 0 and 1
# We set aside 20% of the training set for validation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# We generate an image-label pair for the training set as follows
face_shape_train_gen = datagen.flow_from_dataframe(
    dataframe=face_shape_train, 
    directory="img/",
    x_col="file_name",
    y_col="face_shape",
    class_mode="sparse",
    target_size=(500,500),
    batch_size=32,
    subset="training"
    )

# We generate an image-label pair for the validation set as follows
face_shape_val_gen = datagen.flow_from_dataframe(
    dataframe=face_shape_train, 
    directory="img/",
    x_col="file_name",
    y_col="face_shape",
    class_mode="sparse",
    target_size=(500,500),
    batch_size=32,
    subset="validation"
    )

# We generate an image-label pair for the gender test set as follows
# We set batch_size = size of test set
face_shape_test_gen = datagen.flow_from_dataframe(
    dataframe=face_shape_test, 
    directory="img/",
    x_col="file_name",
    y_col="face_shape",
    class_mode="sparse",
    target_size=(500,500),
    batch_size=len(face_shape_test)
    )

/tensorflow-2.1.0/python3.6/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 14 invalid image filename(s) in x_col="file_name". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 6389 validated image filenames belonging to 5 classes.
Found 1597 validated image filenames belonging to 5 classes.
Found 1996 validated image filenames belonging to 5 classes.


/tensorflow-2.1.0/python3.6/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 4 invalid image filename(s) in x_col="file_name". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [0]:
# Create two ImageDataGenerator objects for the eye color dataset in a similar way

# https://forums.fast.ai/t/split-data-using-fit-generator/4380/4
# for validation split

# We rescale to ensure RGB values fall between 0 and 1
# We set aside 20% of the training set for validation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# We generate an image-label pair for the training set as follows
eye_color_train_gen = datagen.flow_from_dataframe(
    dataframe=eye_color_train, 
    directory="img/",
    x_col="file_name",
    y_col="eye_color",
    class_mode="sparse",
    target_size=(250,250),
    batch_size=32,
    subset="training"
    )

# We generate an image-label pair for the validation set as follows
eye_color_val_gen = datagen.flow_from_dataframe(
    dataframe=eye_color_train, 
    directory="img/",
    x_col="file_name",
    y_col="eye_color",
    class_mode="sparse",
    target_size=(250,250),
    batch_size=32,
    subset="validation"
    )

# We generate an image-label pair for the gender test set as follows
# We set batch_size = size of test set
eye_color_test_gen = datagen.flow_from_dataframe(
    dataframe=eye_color_test, 
    directory="img/",
    x_col="file_name",
    y_col="eye_color",
    class_mode="sparse",
    target_size=(250,250),
    batch_size=len(eye_color_test)
    )

/tensorflow-2.1.0/python3.6/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 14 invalid image filename(s) in x_col="file_name". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 6389 validated image filenames belonging to 5 classes.
Found 1597 validated image filenames belonging to 5 classes.
Found 1996 validated image filenames belonging to 5 classes.


/tensorflow-2.1.0/python3.6/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 4 invalid image filename(s) in x_col="file_name". These filename(s) will be ignored.
  .format(n_invalid, x_col)


## Building and training the models



### Multi-layer Perceptron (MLP)

In [0]:
# Creating a classification MLP with two hidden layers
# We are using the Sequential API which creates a stack of layers
# in which the input flows through one after the other

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Instantiate Sequential API
mlp_model = keras.models.Sequential()

# This flattens the 218x178x3 input into a 1D tensor
mlp_model.add(keras.layers.Flatten(input_shape=(218,178,3)))

# This adds a fully connected layer with 300 neurons using the ReLU
# activation function
mlp_model.add(keras.layers.Dense(300, activation="relu"))

# This adds a fully connected layer with 100 neurons using the ReLU
# activation function
mlp_model.add(keras.layers.Dense(100, activation="relu"))

# This creates the output layer.
mlp_model.add(keras.layers.Dense(5, activation="softmax"))

In [0]:
# We now compile the MLP model to specify the loss function
# and the optimizer to use (SGD)

mlp_model.compile(loss="sparse_categorical_crossentropy", # b/c of exclusive, sparse outputs
                  optimizer='sgd', # We use SGD to optimise the ANN
                  metrics=["accuracy"] # Used for classifiers
                  ) 

In [0]:
type(gender_train_gen.samples)

int

In [40]:
# Training and evaluating the MLP model on the gender dataset
gender_history = mlp_model.fit(
    gender_train_gen,
    steps_per_epoch=gender_train_gen.samples // 32,
    validation_data=gender_val_gen,
    validation_steps=gender_val_gen.samples // 32,
    epochs=2
    )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 25 steps
Epoch 1/2
100/100 [==============================] - 1866s 19s/step - loss: 0.9648 - accuracy: 0.5400 - val_loss: 0.6951 - val_accuracy: 0.5575
Epoch 2/2
100/100 [==============================] - 10s 102ms/step - loss: 0.6540 - accuracy: 0.6484 - val_loss: 0.6522 - val_accuracy: 0.7138


In [48]:
gender_history.history.get('val_accuracy')[-1]

0.71375

In [47]:
mlp_model.evaluate(gender_test_gen)

KeyboardInterrupt: ignored

In [0]:
# Training and evaluating the MLP model on the gender dataset
smiling_history = mlp_model.fit(
    smiling_train_gen,
    steps_per_epoch=smiling_train_gen.samples // 32,
    validation_data=smiling_val_gen,
    validation_steps=smiling_val_gen.samples // 32,
    epochs=10
    )

In [45]:
# Convert gender test generator into numpy arrays
# https://stackoverflow.com/questions/42284873/assign-imagedatagenerator-result-to-numpy-array
def get_X_y_test_sets(test_gen):
    itr = gender_test_gen
    print("Splitting ImageDataGenerator. This may take a while...")
    gender_X_test, gender_y_test = itr.next()
    print("Splitting complete.")
    return gender_X_test, gender_y_test

# Get indices of wrongfully misclassified test set
# https://stackoverflow.com/questions/39300880/how-to-find-wrong-prediction-cases-in-test-set-cnns-using-keras
def get_wrong_indices(X_test, y_test):
    incorrects = np.asarray(np.nonzero(cnn_model.predict(X_test).argmax(axis=-1).reshape((-1,)) != y_test))
    incorrects = incorrects.T.flatten()
    return incorrects

# This returns an array which contains the predictions for the misclassified images
def get_incorrect_preds(X_test, y_test, incorrects):
    incorrect_preds = []

    for incorrect in np.nditer(incorrects):
        probs_pred = cnn_model.predict(gender_X_test[incorrect:incorrect+1])
        incorrect_pred = probs_pred.argmax(axis=-1)
        incorrect_preds.append(incorrect_pred)

    incorrect_preds = np.asarray(incorrect_preds)
    incorrect_preds = incorrect_preds.T.flatten()
    incorrect_preds = incorrect_preds.astype(float)
    return incorrect_preds

# This returns an array which contains the actual labels for the misclassified images
def get_actual_labels(X_test, y_test, incorrects):
    actual_labels = []

    for incorrect in np.nditer(incorrects):
        actual_label = y_test[incorrect]
        actual_labels.append(actual_label)

    actual_labels = np.asarray(actual_labels)
    actual_labels = actual_labels.T.flatten()
    actual_labels = actual_labels.astype(float)
    return actual_labels

# This returns an array which contains the individual losses for the misclassified images
def get_incorrect_losses(X_test, y_test, incorrects):
    incorrect_losses = [] 

    for incorrect in np.nditer(incorrects):
        loss = cnn_model.evaluate(X_test[incorrect:incorrect+1], y_test[incorrect:incorrect+1], verbose=0)
        incorrect_losses.append(loss[0])

    incorrect_losses = np.asarray(incorrect_losses)
    incorrect_losses = incorrect_losses.astype(float)
    return incorrect_losses

def get_probs_correct_label(X_test, y_test, incorrects):
# This returns an array which contains the probabilities of the actual label
# for the misclassified images
    probs_correct_label = []

    for incorrect in np.nditer(incorrects):
        prob_correct_label = cnn_model.predict(gender_X_test[incorrect:incorrect+1])
        probs_correct_label.append(prob_correct_label[0,int(gender_y_test[incorrect])])

    probs_correct_label = np.asarray(probs_correct_label)
    probs_correct_label = probs_correct_label.astype(float)
    return probs_correct_label

def create_loss_pred_data(X_test, y_test):
    incorrects = get_wrong_indices(X_test, y_test)
    incorrect_preds = get_incorrect_preds(X_test, y_test, incorrects)
    actual_labels = get_actual_labels(X_test, y_test, incorrects)
    incorrect_losses = get_incorrect_losses(X_test, y_test, incorrects)
    probs_correct_label = get_probs_correct_label(X_test, y_test, incorrects)

    # This joins together the indices of incorrectly misclassified images, their losses and
    # actual label probabilities into a numpy array. It is then sorted in descending order
    loss_pred_data = np.column_stack((incorrects.astype(float), 
                                      incorrect_preds, 
                                      actual_labels, 
                                      incorrect_losses, 
                                      probs_correct_label))
    loss_pred_data = loss_pred_data[np.argsort(loss_pred_data[:,3])[::-1]]
    return loss_pred_data

# This plots a single misclassified image alongside its predicted label,
# its actual label, the error rate for the image and the probability given
# to the actual label
def plot_image(i, loss_pred_data, img_data):
    plt.imshow(img_data[int(loss_pred_data[i,0])])
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.xlabel("{}/{} {:0.2f} {:0.2f}".format(int(loss_pred_data[i,1]),
                                              int(loss_pred_data[i,2]),
                                              loss_pred_data[i,3],
                                              loss_pred_data[i,4]))

# This plots the images which have been the most misclassified when running the
# model on the test set. Inspired by the plot_top_losses function in the fastai
# library
def plot_top_losses(X_test, y_test, num_rows, num_cols):
    loss_pred_data = create_loss_pred_data(gender_X_test, gender_y_test)
    num_images = num_rows*num_cols
    plt.figure(figsize=(2*num_cols, 2*num_rows))
    for i in range(num_images):
        plt.subplot(num_rows, num_cols, i+1)
        plot_image(i, loss_pred_data, X_test)
    plt.tight_layout()
    return plt.show()

gender_X_test, gender_y_test = get_X_y_test_sets(gender_test_gen)

plot_top_losses(gender_X_test, gender_y_test, 3, 3)

Splitting ImageDataGenerator. This may take a while...


KeyboardInterrupt: ignored

In [0]:
# Training and evaluating the MLP model on the smiling dataset
mlp_smiling_history = mlp_model.fit(
    smiling_train_gen,
    steps_per_epoch=smiling_train_gen.samples // 32,
    validation_data=smiling_val_gen,
    validation_steps=smiling_val_gen.samples // 32,
    epochs=30
    )

In [0]:
get_wrong_indices(gender_X_test, gender_y_test)

array([], dtype=int64)

### ResNet50

In [0]:
base_model = keras.applications.resnet_v2.ResNet50V2(weights="imagenet",
                                                     include_top=False,
                                                     input_shape=(500, 500, 3))
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(5, activation="softmax")(avg)
resnet50v2_model = keras.Model(inputs=base_model.input, outputs=output)

94674944/94668760 [==============================] - 4s 0us/step


In [0]:
for layer in base_model.layers:
    layer.trainable = False

In [0]:
optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)

resnet50v2_model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
resnet50v2history = resnet50v2_model.fit(eye_color_train_gen, epochs=5, validation_data=eye_color_val_gen)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 200 steps, validate for 50 steps
Epoch 1/5
200/200 [==============================] - 79s 395ms/step - loss: 5.6075 - accuracy: 0.4376 - val_loss: 6.8243 - val_accuracy: 0.2248
Epoch 2/5
200/200 [==============================] - 74s 372ms/step - loss: 1.7246 - accuracy: 0.5934 - val_loss: 3.2253 - val_accuracy: 0.2905
Epoch 3/5
200/200 [==============================] - 74s 372ms/step - loss: 1.0661 - accuracy: 0.6652 - val_loss: 3.2229 - val_accuracy: 0.2837
Epoch 4/5
200/200 [==============================] - 74s 371ms/step - loss: 0.9344 - accuracy: 0.6840 - val_loss: 3.1006 - val_accuracy: 0.2918
Epoch 5/5
200/200 [==============================] - 75s 374ms/step - loss: 0.8265 - accuracy: 0.7007 - val_loss: 3.1270 - val_accuracy: 0.2899


### Convolutional Neural Network (CNN)

In [0]:
# We create a simple CNN architecture for image classification
# Architecture from:
# https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb#scrollTo=L1WtoaOHVrVh

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D

cnn_model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(218,178,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(2, activation='softmax')
    ])

In [0]:
# We now compile the CNN model to specify the loss function
# and the optimizer to use (Adam)
cnn_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
    )

In [0]:
# Training and evaluating the CNN model on the gender dataset
cnn_gender_history = cnn_model.fit(
    gender_train_gen,
    steps_per_epoch=gender_train_gen.samples // 32,
    validation_data=gender_val_gen,
    validation_steps=gender_val_gen.samples // 32,
    epochs=10
    )

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 100 steps, validate for 25 steps
Epoch 1/10
100/100 [==============================] - 1129s 11s/step - loss: 0.5223 - accuracy: 0.7519 - val_loss: 0.3748 - val_accuracy: 0.8450
Epoch 2/10
100/100 [==============================] - 10s 102ms/step - loss: 0.2401 - accuracy: 0.9038 - val_loss: 0.2674 - val_accuracy: 0.8988
Epoch 3/10
100/100 [==============================] - 10s 101ms/step - loss: 0.1672 - accuracy: 0.9359 - val_loss: 0.2755 - val_accuracy: 0.8963
Epoch 4/10
100/100 [==============================] - 10s 100ms/step - loss: 0.1061 - accuracy: 0.9600 - val_loss: 0.3034 - val_accuracy: 0.9038
Epoch 5/10
100/100 [==============================] - 10s 99ms/step - loss: 0.0587 - accuracy: 0.9812 - val_loss: 0.3039 - val_accuracy: 0.9075
Epoch 6/10
100/100 [==============================] - 10s 101ms/step - loss: 0.0478 - accuracy: 0.9816 - val_loss: 0.3701 - val_accuracy: 0.8900
Epoch 7/10
100/100 [==================

## Next Steps

* Figure out the principles for building a good MLP and CNN and  architecture for image classification
  * Do experiment on distributing hidden layers evenly, 25/75, 75/25 split.
* Read chapter on training deep neural nets
* Figure out how to do learning rate finder and one cycle learning
* Do literature review
* Write the code in a format that works for submission